## Task 1: Create a "Matrix Operations Tool" using Python and the NumPy library. The application should allow users to input matrices and perform operations like addition, subtraction, multiplication, transpose, and determinant calculation. Include an interactive interface to display results in a structured format.

In [ ]:
import numpy as np
import ipywidgets as widgets
from IPython.display import display, HTML, Markdown, clear_output

display(HTML("""
<style>
.matrix-app .card {
    background: #fff;
    border-radius: 12px;
    box-shadow: 0 4px 12px rgba(0,0,0,0.1);
    margin-bottom: 20px;
}
.matrix-app .card-header {
    background: #2c3e50;
    color: #fff;
    font-weight: bold;
    padding: 10px 15px;
    border-radius: 12px 12px 0 0;
    font-size: 1.1em;
}
.matrix-app .card-body {
    padding: 15px;
}
.matrix-app .compute-btn > button {
    background: #1abc9c !important;
    color: white !important;
    font-weight: bold;
    border: none;
    border-radius: 25px;
    padding: 12px 30px;
    box-shadow: 0 4px 8px rgba(0,0,0,0.15);
    transition: transform .2s;
}
.matrix-app .compute-btn > button:hover {
    background: #16a085 !important;
    transform: scale(1.05);
}
.matrix-app .result-panel {
    background: #ecf0f1;
    padding: 20px;
    border-radius: 0 0 12px 12px;
    font-family: monospace;
    white-space: pre-wrap;
}
</style>
"""))

def make_grid(rows, cols):
    grid = widgets.GridspecLayout(rows, cols, grid_gap="4px")
    for i in range(rows):
        for j in range(cols):
            grid[i, j] = widgets.FloatText(
                layout=widgets.Layout(width="60px"),
                style={'description_width': '0px'}
            )
    return grid

rows_A = widgets.BoundedIntText(2, min=1, max=8, description="Rows A:")
cols_A = widgets.BoundedIntText(2, min=1, max=8, description="Cols A:")
rows_B = widgets.BoundedIntText(2, min=1, max=8, description="Rows B:")
cols_B = widgets.BoundedIntText(2, min=1, max=8, description="Cols B:")

operation = widgets.Dropdown(
    options=[
        ("Add A + B", "add"), ("Subtract A − B", "sub"),
        ("Multiply A × B", "mul"), ("Transpose A", "traA"),
        ("Transpose B", "traB"), ("Determinant A", "detA"),
        ("Determinant B", "detB")
    ],
    description="Operation:",
    style={'description_width': 'initial'}
)

grid_A = make_grid(rows_A.value, cols_A.value)
grid_B = make_grid(rows_B.value, cols_B.value)

acc_A = widgets.Accordion([widgets.VBox([rows_A, cols_A, grid_A])])
acc_A.set_title(0, "Configure Matrix A")

acc_B = widgets.Accordion([widgets.VBox([rows_B, cols_B, grid_B])])
acc_B.set_title(0, "Configure Matrix B")

def rebuild_A(_):
    global grid_A
    grid_A = make_grid(rows_A.value, cols_A.value)
    acc_A.children = [widgets.VBox([rows_A, cols_A, grid_A])]
    acc_A.set_title(0, "Configure Matrix A")

def rebuild_B(_):
    global grid_B
    grid_B = make_grid(rows_B.value, cols_B.value)
    acc_B.children = [widgets.VBox([rows_B, cols_B, grid_B])]
    acc_B.set_title(0, "Configure Matrix B")

rows_A.observe(rebuild_A, 'value')
cols_A.observe(rebuild_A, 'value')
rows_B.observe(rebuild_B, 'value')
cols_B.observe(rebuild_B, 'value')

def toggle_B(change):
    acc_B.layout.display = ('block'
        if change['new'] in ('add','sub','mul','traB','detB') else 'none')

operation.observe(toggle_B, 'value')
toggle_B({'new': operation.value})

config_card = widgets.VBox([
    widgets.HTML("<div class='card-header'>Configuration</div>"),
    widgets.HTML("<div class='card-body'>") ,
    acc_A, acc_B, operation,
    widgets.HTML("</div>")
])
config_card.add_class('card')

compute_btn = widgets.HBox([widgets.Box()], layout={'justify_content':'center'})
compute_btn = widgets.HBox([widgets.Button(description="Compute", button_style='')])
compute_btn.children[0].add_class('compute-btn')
compute_btn.children[0].description = "Compute"
compute_btn.children[0].button_style = ''
compute_btn.children[0].layout.margin = '10px auto'

out = widgets.Output()
result_card = widgets.VBox([
    widgets.HTML("<div class='card-header'>Result</div>"),
    widgets.HTML("<div class='result-panel'>"),
    out,
    widgets.HTML("</div>")
])
result_card.add_class('card')

def on_compute(_):
    out.clear_output()
    A = np.array([[grid_A[i,j].value for j in range(cols_A.value)]
                  for i in range(rows_A.value)])
    B = None
    if acc_B.layout.display == 'block':
        B = np.array([[grid_B[i,j].value for j in range(cols_B.value)]
                      for i in range(rows_B.value)])
    try:
        if operation.value == 'add':
            R = A + B
        elif operation.value == 'sub':
            R = A - B
        elif operation.value == 'mul':
            R = A.dot(B)
        elif operation.value == 'traA':
            R = A.T
        elif operation.value == 'traB':
            R = B.T
        elif operation.value == 'detA':
            R = np.linalg.det(A)
        elif operation.value == 'detB':
            R = np.linalg.det(B)
        else:
            raise ValueError("Unknown op")
    except Exception as e:
        R = f"Error: {e}"

    with out:
        if isinstance(R, np.ndarray):
            txt = "\n".join("  ".join(f"{v:.4g}" for v in row) for row in R)
        else:
            txt = str(R)
        print(txt)

compute_btn.children[0].on_click(on_compute)

app = widgets.VBox([config_card, compute_btn, result_card])
app.add_class('matrix-app')
display(app)
